In [1]:
import os
from pathlib import Path
TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Building Sensitivity Analisys


The aim of this tutorial is to provide a complete workflow for building thermal simulation sensitivity analysis using
[__EnergyPlus__](https://energyplus.net) and __energytoot__

## Introduction

Sensitivity Analysis methods are mathematical method that quantify the impact of and uncertain parameters on a specific metrics.
Various methods exists such as Morris or Sobol.

During building conception workflow, sensitivity analysis can have various benefits:
- Screen out a number of irrelevant conception variables to focus on the important ones (does the solar absorption of the partition glass-wool have a relevant impact on building heat needs ?)
- Sort  relevant uncertain parameter by influence on the observed metrics.
- Help you quantify the relative importance of the modeled physical phenomenons

In this example, the use case is an old building retrofitting. The objective is to insulate the south facade using double skin.

|               Figure 1: Building picture               |            Figure 2: Building thermal model            |
|:------------------------------------------------------:|:------------------------------------------------------:|
| <img src="resources/building_photo.png"  height="300"> | <img src="resources/building_model.png"  height="300"> |

The designer wants to know the impact of the following variables on the building **heat needs** and on the **thermal comfort**:
- Double skin glazing thermal properties : Solar Heat Gain Coefficient (SHGC), thermal conductivity coefficient ($U_{value}$)
- Envelope glazing thermal properties : Solar Heat Gain Coefficient (SHGC), thermal conductivity coefficient ($U_{value}$)
- Envelope insulation thickness
- Air infiltration coefficient Q4Pa [m<sup>3</sup>/h.m² @4Pa]



To answer these questions we will use an EnergyPlus building model and energytool class to perform Morris & Sobol sensitivity analysis

## Building modeling

In energytool, the <code>Building</code> class is used to simulate HVAC systems through pre-process and post process methods.
The <code>Building</code> holds and idf file. The user specify hvac system using the <code>system</code> module.

In [2]:
from energytool.building import Building

The path of the idd file of EnergyPlus must be given to the <code>Building</code> class.
Be careful idf file E+ version and idd file version must match

In [3]:
Building.set_idd(Path(r'C:\EnergyPlusV9-4-0'))

Now we instantiate a building with an idf file representing the building thermal model.
The idf can be generated manually or using a software (Openstudio, DesignBuilder).
Keep in mind that the main advantage of the energytool <code>Building</code> class, is to simplify hvac system modeling using pre-process and post-process methods.
Thus, we recommend using <code>IdealLoadAirSystem</code> to model HVAC and Domestic Hot Water production (DHW).

In [4]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/tuto_as.idf")

The <code>infos()</code> method display information on the building object

In [5]:
building.infos()

==Building==

Number of occupants : 80.21
Building surface : 750.0895999999999 m²
Building volume : 2299.8318 m3
Zone number : 5

==HVAC systems==

Heating systems : []
Cooling systems : []
Ventilation system : []
Artificial lighting system : []
DHW production : []
PV production : []
Others : []


Now it is time to specify the building hvac equipments.
We will use the one present in the <code>system</code> module.
Note that you can use custom class as long as they contain a <code>pre_process()</code> and a <code>post_process()</code> methods

In this example, we will only use a boiler with a cop of 1 as we want to work with building heating needs.
For more information on building system, see the dedicated tutorial.

In [6]:
import energytool.system as st

In [7]:
building.heating_system["Main_heater"] = st.HeaterSimple(
    name="IdealBoiler",
    building=building,
    zones='*',
    cop=1
)

It is now time to specify the parameters we are uncertain about.
We use the class <code>UncertainParameter</code> from the <code>energytool.parameter</code> module.

In [8]:
from energytool.parameter import UncertainParameter

A <code>list</code> of <code>UncertainParameter</code> is created.
We detail the configuration for the first object. It corresponds to the SHGC of the external windows.
We specify a relative uncertainty of +- 10% of the idf value:

<pre><code>
UncertainParameter(
    name="SHGC_ext_windows",
    absolute=False,
    bounds=[0.9, 1.1],
    building=building,
    idf_parameters=[dict(
        idf_object="WindowMaterial:SimpleGlazingSystem",
        names='Simple DSF_ext_south_glazing - 1002',
        field='Solar_Heat_Gain_Coefficient',
    )]
</code></pre>

- <code>absolute</code>: if set to <code>True</code> the <code>bounds</code> holds the true values of the parameter. If set to <code>False</code> specify relative the <code>bounds</code>. The nominal value is the one of the idf.
- <code>bounds</code>: the minimum and maximum value of the uncertain parameter.
- <code>idf_parameters</code>: a python the <code>dict</code> with the following keys:
    - <code>idf_object</code> idf object category
    - <code>names</code> a specific idf object name. a list of idf objects names. Default '*' indicates it applies to all specified <code>idf_object</code>
    - <code>field</code> uncertain numerical field


In [11]:
uncertain_param_list = [
    UncertainParameter(
        name="SHGC_ext_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_ext_south_glazing - 1002',
            field='Solar_Heat_Gain_Coefficient',
        )]
    ),
    UncertainParameter(
        name="UFactor_ext_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_ext_south_glazing - 1002',
            field='UFactor',
        )]
    ),
    UncertainParameter(
        name="SHGC_int_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_int_south_glazing - 1001',
            field='Solar_Heat_Gain_Coefficient',
        )]
    ),
    UncertainParameter(
        name="UFactor_int_windows",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="WindowMaterial:SimpleGlazingSystem",
            names='Simple DSF_int_south_glazing - 1001',
            field='UFactor',
        )]
    ),
    UncertainParameter(
        name="Wall_insulation_thickness",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="Material",
            names='Wall_insulation_.1',
            field='Thickness',
        )]
    ),
    UncertainParameter(
        name="Cracks",
        absolute=False,
        bounds=[0.9, 1.1],
        building=building,
        idf_parameters=[dict(
            idf_object="AirflowNetwork:MultiZone:Surface:Crack",
            names='*',
            field='Air_Mass_Flow_Coefficient_at_Reference_Conditions',
        )]
    ),
]

We import the sensitivity analysis class <code>SAnalysis</code>
As a minimal configuration, it requires the <code>Building</code> instance, en sensitivity analysis method and the previously defined list of uncertain parameter.

To screen out parameters or to have a first estimation of the uncertain parameters rank without running too many simulation, it is often a good idea to us the Morris method.

In [2]:
from energytool.sensitivity import SAnalysis

In [3]:
sa_analysis = SAnalysis(
    building=building,
    sensitivity_method="Morris",
    parameters=uncertain_param_list,
)

NameError: name 'building' is not defined

In [4]:
sa_analysis.draw_sample(n=15)

NameError: name 'sa_analysis' is not defined

In [15]:
sa_analysis.run_simulations(epw_file_path=Path(TUTORIAL_DIR) / r"resources/FRA_Bordeaux.075100_IWEC.epw")

In [16]:
sa_analysis.analyze()

In [17]:
from energytool.sensitivity import plot_morris_scatter

In [19]:
plot_morris_scatter(salib_res=sa_analysis.sensitivity_results, title='Building heat needs', unit='J', autosize=False)

In [20]:
sa_analysis_sob = SAnalysis(
    building=building,
    sensitivity_method="Sobol",
    parameters=uncertain_param_list,
)

In [28]:
sa_analysis_sob.draw_sample(n=2**5)

In [29]:
sa_analysis_sob.run_simulations(epw_file_path=Path(TUTORIAL_DIR) / r"resources/FRA_Bordeaux.075100_IWEC.epw")

In [30]:
sa_analysis_sob.analyze()

In [32]:
from energytool.sensitivity import plot_sobol_st_bar

In [33]:
plot_sobol_st_bar(sa_analysis_sob.sensitivity_results)

In [36]:
sa_analysis_sob.sensitivity_results['ST'].sum()

0.9424838946599576

In [38]:
sa_analysis_sob.sensitivity_results.to_df().to_csv(Path(r"C:\Users\bdurandestebe\Documents\37-FIAB_GPE\sobol_save.csv"))

AttributeError: 'list' object has no attribute 'to_csv'

In [45]:
sa_analysis_sob.sensitivity_results.to_df()[0].to_csv(Path(r"C:\Users\bdurandestebe\Documents\37-FIAB_GPE\sobol_ordre_total_save.csv"))

In [46]:
sa_analysis_sob.sensitivity_results.to_df()[1].to_csv(Path(r"C:\Users\bdurandestebe\Documents\37-FIAB_GPE\sobol_ordre_1_save.csv"))

In [47]:
sa_analysis_sob.sensitivity_results.to_df()[2].to_csv(Path(r"C:\Users\bdurandestebe\Documents\37-FIAB_GPE\sobol_ordre_2_save.csv"))